In [1]:
%matplotlib inline

This is testing the Nipype DTI-TK interfaces according to the following tutorial: http://dti-tk.sourceforge.net/pmwiki/pmwiki.php?n=Documentation.Registration

Note: some text excerpts are copied directly from the DTI-TK documentation at the address, above, and have been adapted for the Nipype implementation

DTI-TK developed by Gary Hui Zhang, gary.zhang@ucl.ac.uk
You are running Release 2.3.3. For additional help, visit http://dti-tk.sf.net

In [2]:
import os
from glob import glob
import nipype.interfaces.dtitk as dtitk

/Users/kesshijordan/anaconda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


#not helping
%load_ext autoreload
%autoreload 2

#this isn't working since I installed in developer mode
import importlib as imp
imp.reload(dtitk)

In [3]:
#basepath = '/Users/kesshijordan/ref_data/interscan_dys/test_dtitk_2/sandbox/'
basepath = '.'
fixed_tensor = os.path.join(basepath, 'Ptensor.nii.gz')
moving_tensor = os.path.join(basepath, 'Ttensor.nii.gz')
fixed_vol = './Ptensor_tr.nii.gz'
moving_vol = './Ttensor_tr.nii.gz'

# Let's see what we have

## Get information about each volume 

In [4]:
! VolumeInfo 'Ptensor.nii.gz'

NIFTI Intent Code: NIFTI_INTENT_SYMMATRIX
NIFTI Orientation Code: RPI
Volume Info of Ptensor.nii.gz
size: 65x88x69, voxel size: 2x2x2, origin: [-0, -0, -0]


In [5]:
! VolumeInfo 'Ttensor.nii.gz'

NIFTI Intent Code: NIFTI_INTENT_SYMMATRIX
NIFTI Orientation Code: RPI
Volume Info of Ttensor.nii.gz
size: 62x80x60, voxel size: 2.2x2.2x2.2, origin: [-0, -0, -0]


# Preprocessing: 

(not sure why there's redundant functionality in TVAdjustVoxelspace and TVResample)

KMJ Note: 
- The tutorial recommends moving all volumes to origin [0,0,0] before processing. 
- If you want to do a diffeomorphic registration, make sure you resample such that the dimension 0 is a power of 2

## Adjust Voxel Space

- Output: New volume that has a new voxel size and/or origin (not: if output not specified, then the input volume is modified)
- Usage: TVAdjustVoxelspace -in tensor.nii.gz -out output.nii.gz -target target_tensor.nii.gz -vsize x y z -origin x y z

So you need to provide as input

- target: the filename of the template tensor you want matched in origin and voxel size

OR

- voxel size (vsize): the x y z sizes
    - DEFAULT not automatically passed
- origin: the x y z coordinates of the intended origin
    - DEFAULT {x,y,z} = {0,0,0}

NOTE: if you provide both a target tensor file and specify vsize and/or origin, then the tensor file space will be used

If you want to specify an output file, then you can override the default, which adds "_avs" to the end of the input file

### Tensor Volume

Usage of TVAdjustVoxelspace : 

 - -in            type: string(s)    required: Y	
 - -out           type: string(s)    required: N	by default use the input file name
 - -target        type: string(s)    required: N	output volume origin read from the target volume if specified
 - -vsize         type: double(s)    required: N	
 - -origin        type: double(s)    required: N	

In [6]:
tv_avs = dtitk.utils.TVAdjustVoxSpTask()
tv_avs.inputs.in_file = fixed_tensor
tv_avs.inputs.origin = (0,0,0)
tv_avs.run()

180326-12:38:39,999 interface INFO:
	 stdout 2018-03-26T12:38:39.999468:Reading ./Ptensor.nii.gz ... Done in 0.036603s
180326-12:38:40,2 interface INFO:
	 stderr 2018-03-26T12:38:40.002819:WARNING: qform not defined
180326-12:38:40,4 interface INFO:
	 stderr 2018-03-26T12:38:40.002819:WARNING: assumed as RPI
180326-12:38:40,73 interface INFO:
	 stdout 2018-03-26T12:38:40.073820:input voxelspace : size: 65x88x69, voxel size: 2x2x2, origin: [-0, -0, -0]
180326-12:38:40,75 interface INFO:
	 stdout 2018-03-26T12:38:40.073820:output voxelspace : size: 65x88x69, voxel size: 2x2x2, origin: [0, 0, 0]
180326-12:38:40,424 interface INFO:
	 stdout 2018-03-26T12:38:40.423885:Writing Ptensor_avs.nii.gz ... Done in 0.249084s


In [7]:
tv_avs = dtitk.utils.TVAdjustVoxSpTask()
tv_avs.inputs.in_file = moving_tensor
tv_avs.inputs.origin = (0,0,0)
tv_avs.run()

180326-12:38:40,623 interface INFO:
	 stdout 2018-03-26T12:38:40.623003:Reading ./Ttensor.nii.gz ... Done in 0.03331s
180326-12:38:40,628 interface INFO:
	 stderr 2018-03-26T12:38:40.628161:WARNING: qform not defined
180326-12:38:40,629 interface INFO:
	 stderr 2018-03-26T12:38:40.628161:WARNING: assumed as RPI
180326-12:38:40,679 interface INFO:
	 stdout 2018-03-26T12:38:40.679168:input voxelspace : size: 62x80x60, voxel size: 2.2x2.2x2.2, origin: [-0, -0, -0]
180326-12:38:40,680 interface INFO:
	 stdout 2018-03-26T12:38:40.679168:output voxelspace : size: 62x80x60, voxel size: 2.2x2.2x2.2, origin: [0, 0, 0]
180326-12:38:40,975 interface INFO:
	 stdout 2018-03-26T12:38:40.975356:Writing Ttensor_avs.nii.gz ... Done in 0.209124s


### Scalar Volume

Usage of SVAdjustVoxelspace : 

 - -in            type: string(s)    required: Y	
 - -out           type: string(s)    required: N	by default use the input file name
 - -target        type: string(s)    required: N	output volume origin read from the target volume if specified
 - -vsize         type: double(s)    required: N	
 - -origin        type: double(s)    required: N	

In [8]:
sv_avs = dtitk.utils.SVAdjustVoxSpTask()
sv_avs.inputs.in_file = fixed_vol
sv_avs.inputs.target_file = moving_vol
sv_avs.run()

print(sv_avs.cmdline)

180326-12:38:41,155 interface INFO:
	 stdout 2018-03-26T12:38:41.155737:Reading ./Ptensor_tr.nii.gz ... Done in 0.007118s
180326-12:38:41,164 interface INFO:
	 stdout 2018-03-26T12:38:41.164352:input voxelspace : size: 65x88x69, voxel size: 2x2x2, origin: [-0, -0, -0]
180326-12:38:41,165 interface INFO:
	 stdout 2018-03-26T12:38:41.164352:reading output volume origin from ./Ttensor_tr.nii.gz
180326-12:38:41,167 interface INFO:
	 stdout 2018-03-26T12:38:41.167834:output voxelspace : size: 65x88x69, voxel size: 2.2x2.2x2.2, origin: [-0, -0, -0]
180326-12:38:41,222 interface INFO:
	 stdout 2018-03-26T12:38:41.222234:Writing Ptensor_tr_avs.nii.gz ... Done in 0.050109s
SVAdjustVoxelspace -in ./Ptensor_tr.nii.gz -out Ptensor_tr_avs.nii.gz -target ./Ttensor_tr.nii.gz


In [9]:
sv_avs.inputs.in_file = moving_vol
sv_avs.inputs.target_file = fixed_vol
sv_avs.run()

print(sv_avs.cmdline)

180326-12:38:41,415 interface INFO:
	 stdout 2018-03-26T12:38:41.415720:Reading ./Ttensor_tr.nii.gz ... Done in 0.006043s
180326-12:38:41,424 interface INFO:
	 stdout 2018-03-26T12:38:41.424059:input voxelspace : size: 62x80x60, voxel size: 2.2x2.2x2.2, origin: [-0, -0, -0]
180326-12:38:41,425 interface INFO:
	 stdout 2018-03-26T12:38:41.424059:reading output volume origin from ./Ptensor_tr.nii.gz
180326-12:38:41,427 interface INFO:
	 stdout 2018-03-26T12:38:41.427823:output voxelspace : size: 62x80x60, voxel size: 2x2x2, origin: [-0, -0, -0]
180326-12:38:41,483 interface INFO:
	 stdout 2018-03-26T12:38:41.483144:Writing Ttensor_tr_avs.nii.gz ... Done in 0.041884s
SVAdjustVoxelspace -in ./Ttensor_tr.nii.gz -out Ttensor_tr_avs.nii.gz -target ./Ptensor_tr.nii.gz


## Resample 

- Output: New volume that has a new voxel size and/or origin (not: if output not specified, then the input volume is modified)
- Usage: TVAdjustVoxelspace -in tensor.nii.gz -out output.nii.gz -target target_tensor.nii.gz -vsize x y z -origin x y z

So you need to provide as input

- target: the filename of the template tensor you want matched

OR the specific parameters you want to change

- array size (size): the dimensions of the image
- voxel size (vsize): the x y z sizes
    - DEFAULT not automatically passed
- origin: the x y z coordinates of the intended origin
    - DEFAULT {x,y,z} = (0,0,0)

NOTE: if you provide both a target tensor file and specify vsize and/or origin, then the tensor file space will be used

In [10]:
tv_resample = dtitk.utils.TVResampleTask()
tv_resample.inputs.in_file = fixed_tensor
tv_resample.inputs.origin = (0,0,0)
tv_resample.inputs.array_size= (128,128,64)
tv_resample.run()

print(tv_resample.cmdline)

180326-12:38:41,701 interface INFO:
	 stdout 2018-03-26T12:38:41.700861:interpolation option is LEI
180326-12:38:41,755 interface INFO:
	 stdout 2018-03-26T12:38:41.755745:Reading ./Ptensor.nii.gz ... Done in 0.049159s
180326-12:38:41,760 interface INFO:
	 stderr 2018-03-26T12:38:41.759767:WARNING: qform not defined
180326-12:38:41,764 interface INFO:
	 stderr 2018-03-26T12:38:41.759767:WARNING: assumed as RPI
180326-12:38:41,861 interface INFO:
	 stdout 2018-03-26T12:38:41.861861:input volume ./Ptensor.nii.gz
180326-12:38:41,863 interface INFO:
	 stdout 2018-03-26T12:38:41.861861:size: 65x88x69, voxel size: 2x2x2, origin: [-0, -0, -0]
180326-12:38:41,925 interface INFO:
	 stdout 2018-03-26T12:38:41.925767:output volume specification: size: 128x128x64, voxel size: 2x2x2, origin: [0, 0, 0]
180326-12:38:42,89 interface INFO:
	 stdout 2018-03-26T12:38:42.089423:matching the center of the old and the new voxel spaces
180326-12:38:42,802 interface INFO:
	 stdout 2018-03-26T12:38:42.802066:W

In [11]:
tv_resample.inputs.in_file=moving_tensor
tv_resample.run()

print(tv_resample.cmdline)

180326-12:38:42,955 interface INFO:
	 stdout 2018-03-26T12:38:42.954728:interpolation option is LEI
180326-12:38:42,990 interface INFO:
	 stdout 2018-03-26T12:38:42.990161:Reading ./Ttensor.nii.gz ... Done in 0.035798s
180326-12:38:42,991 interface INFO:
	 stderr 2018-03-26T12:38:42.991629:WARNING: qform not defined
180326-12:38:42,993 interface INFO:
	 stderr 2018-03-26T12:38:42.991629:WARNING: assumed as RPI
180326-12:38:43,47 interface INFO:
	 stdout 2018-03-26T12:38:43.047158:input volume ./Ttensor.nii.gz
180326-12:38:43,48 interface INFO:
	 stdout 2018-03-26T12:38:43.047158:size: 62x80x60, voxel size: 2.2x2.2x2.2, origin: [-0, -0, -0]
180326-12:38:43,97 interface INFO:
	 stdout 2018-03-26T12:38:43.097291:output volume specification: size: 128x128x64, voxel size: 2.2x2.2x2.2, origin: [0, 0, 0]
180326-12:38:43,213 interface INFO:
	 stdout 2018-03-26T12:38:43.213575:matching the center of the old and the new voxel spaces
180326-12:38:43,939 interface INFO:
	 stdout 2018-03-26T12:38:4

In [12]:
sv_resample = dtitk.utils.SVResampleTask()
sv_resample.inputs.in_file = fixed_vol
sv_resample.inputs.origin = (0,0,0)
sv_resample.inputs.array_size= (128,128,64)
sv_resample.run()

print(sv_resample.cmdline)

180326-12:38:44,177 interface INFO:
	 stdout 2018-03-26T12:38:44.177508:Reading ./Ptensor_tr.nii.gz ... Done in 0.008807s
180326-12:38:44,190 interface INFO:
	 stdout 2018-03-26T12:38:44.190463:input volume ./Ptensor_tr.nii.gz
180326-12:38:44,192 interface INFO:
	 stdout 2018-03-26T12:38:44.190463:size: 65x88x69, voxel size: 2x2x2, origin: [-0, -0, -0]
180326-12:38:44,194 interface INFO:
	 stdout 2018-03-26T12:38:44.194073:output volume specification: size: 128x128x64, voxel size: 2x2x2, origin: [0, 0, 0]
180326-12:38:44,195 interface INFO:
	 stdout 2018-03-26T12:38:44.194073:matching the center of the old and the new voxel spaces
180326-12:38:44,434 interface INFO:
	 stdout 2018-03-26T12:38:44.434813:Writing Ptensor_tr_resampled.nii.gz ... Done in 0.063186s
SVResample -size 128 128 64 -in ./Ptensor_tr.nii.gz -origin 0 0 0 -out Ptensor_tr_resampled.nii.gz


In [13]:
sv_resample.inputs.in_file = moving_vol
sv_resample.run()

print(sv_resample.cmdline)

180326-12:38:44,607 interface INFO:
	 stdout 2018-03-26T12:38:44.607352:Reading ./Ttensor_tr.nii.gz ... Done in 0.005805s
180326-12:38:44,613 interface INFO:
	 stdout 2018-03-26T12:38:44.613198:input volume ./Ttensor_tr.nii.gz
180326-12:38:44,617 interface INFO:
	 stdout 2018-03-26T12:38:44.613198:size: 62x80x60, voxel size: 2.2x2.2x2.2, origin: [-0, -0, -0]
180326-12:38:44,622 interface INFO:
	 stdout 2018-03-26T12:38:44.622323:output volume specification: size: 128x128x64, voxel size: 2.2x2.2x2.2, origin: [0, 0, 0]
180326-12:38:44,627 interface INFO:
	 stdout 2018-03-26T12:38:44.622323:matching the center of the old and the new voxel spaces
180326-12:38:44,754 interface INFO:
	 stdout 2018-03-26T12:38:44.753991:Writing Ttensor_tr_resampled.nii.gz ... Done in 0.057097s
SVResample -size 128 128 64 -in ./Ttensor_tr.nii.gz -origin 0 0 0 -out Ttensor_tr_resampled.nii.gz


In [14]:
fixed_tensor_new = 'Ptensor_resampled.nii.gz'
moving_tensor_new = 'Ttensor_resampled.nii.gz'

## Calculate useful maps

### TVtool (NOTE: I haven't implemented all of them.. selected a few that are tested)

Usage of TVtool : 

 -in            type: string(s)    required: Y	
 -out           type: string(s)    required: N	
 -ad            type: flag         required: N	
 -rd            type: flag         required: N	
 -tr            type: flag         required: N	
 -fa            type: flag         required: N	
 -norm          type: flag         required: N	
 -dnorm         type: flag         required: N	
 -tsp           type: flag         required: N	
 -dtsp          type: flag         required: N	
 -dydisp        type: flag         required: N	
 -dycoh         type: flag         required: N	
 -lps           type: flag         required: N	
 -eigs          type: flag         required: N	
 -pdmp          type: flag         required: N	
 -faThreshold   type: double(s)    required: N	default: 0.2 
 -pd            type: flag         required: N	
 -rgb           type: flag         required: N	
 -luminance     type: double(s)    required: N	default: 1 	Adjusting RGB map luminance
 -faWeighting   type: integer(s)   required: N	default: 1 	Scaling by FA (1) or not (0)
 -spd           type: flag         required: N	
 -reorient      type: flag         required: N	
 -trans         type: string(s)    required: N	
 -euler         type: double(s)    required: N	
 -dev           type: flag         required: N	
 -dyadic        type: flag         required: N	
 -index         type: integer(s)   required: N	default: 0 
 -scale         type: double(s)    required: N	
 -add           type: string(s)    required: N	
 -subtract      type: string(s)    required: N	
 -mosaic        type: string(s)    required: N	
 -piece         type: integer(s)   required: N	default: 16 
 -aop           type: string(s)    required: N	
 -mask          type: string(s)    required: N	
 -alternate     type: double(s)    required: N	
 -nan           type: flag         required: N	
 -print         type: string(s)    required: N	
 -vsm           type: string(s)    required: N	
 -sm            type: string(s)    required: N	
 -SMOption      type: string(s)    required: N	default: EDS 	ED, DDS, EDS or GDS
 -orientation   type: string(s)    required: N	
 -replacement   type: string(s)    required: N	

In [15]:
tvt = dtitk.TVtoolTask()
tvt.inputs.in_file = fixed_tensor_new
tvt.inputs.in_flag = 'tr'

tvt_res = tvt.run()
print(tvt.cmdline)

180326-12:38:45,20 interface INFO:
	 stdout 2018-03-26T12:38:45.020764:Reading Ptensor_resampled.nii.gz ... Done in 0.07496s
180326-12:38:45,353 interface INFO:
	 stdout 2018-03-26T12:38:45.353480:Computing the tensor trace (mean diffusion) map ... Done in 0.010405s
180326-12:38:45,434 interface INFO:
	 stdout 2018-03-26T12:38:45.434449:Writing Ptensor_resampled_tr.nii.gz ... Done in 0.051628s
TVtool -in Ptensor_resampled.nii.gz -tr -out Ptensor_resampled_tr.nii.gz


In [16]:
tvt.inputs.in_file = moving_tensor_new

tvt_res = tvt.run()
print(tvt.cmdline)

180326-12:38:45,708 interface INFO:
	 stdout 2018-03-26T12:38:45.707883:Reading Ttensor_resampled.nii.gz ... Done in 0.07s
180326-12:38:45,905 interface INFO:
	 stdout 2018-03-26T12:38:45.905421:Computing the tensor trace (mean diffusion) map ... Done in 0.009531s
180326-12:38:45,957 interface INFO:
	 stdout 2018-03-26T12:38:45.957374:Writing Ttensor_resampled_tr.nii.gz ... Done in 0.035757s
TVtool -in Ttensor_resampled.nii.gz -tr -out Ttensor_resampled_tr.nii.gz


In [17]:
fixed_tr_new = 'Ptensor_resampled_tr.nii.gz'
moving_tr_new = 'Ttensor_resampled_tr.nii.gz'

### SVTool not implemented at this time

### Binarize Image (probably for mask to input to diffeo reg)

Usage:

BinaryThresholdImageFilter inputImage outputImage lower upper inside outside

- lower = lower bound threshold
- upper = upper bound threshold
- inside = value for voxels inside lower/upper bounds
- outside = value for voxels outside lower/upper bounds

KMJ NOTE: I'm not sure if you would want a decimal as the thresholds or inside/outside values. May want to change these to integers if that is unlikely

In [18]:
bt = dtitk.BinThreshTask()
bt.inputs.in_file = fixed_tr_new
bt.inputs.lower_bound = 0.01
bt.inputs.upper_bound=100
bt.inputs.inside_value=1
bt.inputs.outside_value=0
#bt.inputs.out_path = .replace('.nii.gz', '_trbmask.nii.gz')

bt.run()

print(bt.cmdline)

BinaryThresholdImageFilter Ptensor_resampled_tr.nii.gz Ptensor_resampled_tr_thrbin.nii.gz 0.01 100 1 0


In [19]:
bt.inputs.in_file = moving_tr_new
bt.run()

print(bt.cmdline)

BinaryThresholdImageFilter Ttensor_resampled_tr.nii.gz Ttensor_resampled_tr_thrbin.nii.gz 0.01 100 1 0


In [20]:
fixed_mask = 'Ptensor_resampled_tr_thrbin.nii.gz'
moving_mask = 'Ttensor_resampled_tr_thrbin.nii.gz'

# Alignment of DTI Volumes

## Rigid Registration

### dti_rigid_reg

- Output: Rigid alignment of a DTI volume (the subject) to a DTI template
- Usage: dti_rigid_reg template subject SMOption xsep ysep zsep ftol [useInTrans]

So you need to provide as input

- template: the filename of the template
- subject: the filename of the subject
- SMOption: the similarity metric you'd like to use for assessing the quality of alignment 
    - EDS: Euclidean Distance Squared between tensors (DEFAULT)
    - GDS: Geometric Distance Squared between tensors
    - DDS: Deviatoric Distance Squared between deviatoric tensors
    - NMI: Normalized Mutual Information between tensor components
- {x,y,z}sep: the distance between the sample points in the volume for the evaluation of image similarity (mm)
    - DEFAULT {x,y,z} = {2,2,2}
- ftol: the minimum amount of change in the cost function as a fraction of the previous value at the previous iteration for the registration optimization to stop
    - DEFAULT = 0.01
- useInTrans: the optional input to specify whether to initialize with some previously determined transformation.

The output from this command includes the registered image and the transformation that achieves the alignment. If we denote the filename of the subject as \${prefix}.nii.gz, then the filename of the registered image will be \${prefix_aff.nii.gz and that of the transformation will be \${prefix}.aff.

### Test interface

In [21]:
rigid = dtitk.RigidTask()
rigid.inputs.fixed_file = fixed_tensor_new
rigid.inputs.moving_file = moving_tensor_new

rigid_res = rigid.run()
print(rigid.cmdline)

180326-12:38:46,574 interface INFO:
	 stdout 2018-03-26T12:38:46.574616:Rigid Registration of Ttensor_resampled.nii.gz to Ptensor_resampled.nii.gz
180326-12:38:46,578 interface INFO:
	 stdout 2018-03-26T12:38:46.574616:Similarity Measure : EDS
180326-12:38:46,580 interface INFO:
	 stdout 2018-03-26T12:38:46.574616:Reorient Option : FS
180326-12:38:46,675 interface INFO:
	 stdout 2018-03-26T12:38:46.675682:Reading Ptensor_resampled.nii.gz ... Done in 0.086175s
180326-12:38:46,952 interface INFO:
	 stdout 2018-03-26T12:38:46.952175:sampling step size = [2, 2, 2]
180326-12:38:47,307 interface INFO:
	 stdout 2018-03-26T12:38:47.307505:Gaussian smoothing: sigma = [0.736, 0.736, 0.736] ... Done in 0.336s
180326-12:38:47,374 interface INFO:
	 stdout 2018-03-26T12:38:47.374511:Reading Ttensor_resampled.nii.gz ... Done in 0.0568s
180326-12:38:47,909 interface INFO:
	 stdout 2018-03-26T12:38:47.908988:Gaussian smoothing: sigma = [0.645, 0.645, 0.645] ... Done in 0.331s
180326-12:38:48,119 interf

## Affine Registration

### dti_affine_reg

- Output: Affine alignment of a DTI volume (the subject) to a DTI template
- Usage: dti_affine_reg template subject SMOption xsep ysep zsep ftol [useInTrans]

So you need to provide as input

- template: the filename of the template
- subject: the filename of the subject
- SMOption: the similarity metric you'd like to use for assessing the quality of alignment 
    - EDS: Euclidean Distance Squared between tensors
    - GDS: Geometric Distance Squared between tensors
    - DDS: Deviatoric Distance Squared between deviatoric tensors
    - NMI: Normalized Mutual Information between tensor components
- {x,y,z}sep: the distance between the sample points in the volume for the evaluation of image similarity (mm)
- ftol: the minimum amount of change in the cost function as a fraction of the previous value at the previous iteration for the registration optimization to stop
- useInTrans: the optional input to specify whether to initialize with some previously determined transformation.

The output from this command includes the registered image and the transformation that achieves the alignment. If we denote the filename of the subject as \${prefix}.nii.gz, then the filename of the registered image will be \${prefix_aff.nii.gz and that of the transformation will be \${prefix}.aff.

### Test interface

In [22]:
affine = dtitk.AffineTask()
affine.inputs.fixed_file = fixed_tensor_new
affine.inputs.moving_file = moving_tensor_new
#Uncomment this line to use the rigid registration to initialize
affine.inputs.useInTrans = True

affine_res = affine.run()
print(affine.cmdline)

180326-12:38:53,201 interface INFO:
	 stdout 2018-03-26T12:38:53.200936:Affine Registration of Ttensor_resampled.nii.gz to Ptensor_resampled.nii.gz
180326-12:38:53,203 interface INFO:
	 stdout 2018-03-26T12:38:53.203482:Similarity Measure : EDS
180326-12:38:53,205 interface INFO:
	 stdout 2018-03-26T12:38:53.203482:Reorient Option : FS
180326-12:38:53,272 interface INFO:
	 stdout 2018-03-26T12:38:53.272623:Reading Ptensor_resampled.nii.gz ... Done in 0.06749s
180326-12:38:53,470 interface INFO:
	 stdout 2018-03-26T12:38:53.470448:sampling step size = [2, 2, 2]
180326-12:38:53,877 interface INFO:
	 stdout 2018-03-26T12:38:53.877759:Gaussian smoothing: sigma = [0.736, 0.736, 0.736] ... Done in 0.358s
180326-12:38:53,928 interface INFO:
	 stdout 2018-03-26T12:38:53.928369:Reading Ttensor_resampled.nii.gz ... Done in 0.0499s
180326-12:38:54,429 interface INFO:
	 stdout 2018-03-26T12:38:54.429560:Gaussian smoothing: sigma = [0.645, 0.645, 0.645] ... Done in 0.311s
180326-12:38:54,628 interf

## Apply Affine Transform to Tensor Volume

### affineSymTensor3DVolume

Usage of affineSymTensor3DVolume : 

 - -in            type: string(s)    required: Y	
 - -out           type: string(s)    required: Y	
 - -trans         type: string(s)    required: N	specify an input transformation file; parameters input will be ignored
 - -target        type: string(s)    required: N	output volume specification read from the target volume if specified
 - -interp        type: string(s)    required: N	default: LEI 	EI or LEI
 - -reorient      type: string(s)    required: N	default: PPD 	NO, FS or PPD
 - -translation   type: double(s)    required: N	default: 0 0 0 	x y z in mm
 - -euler         type: double(s)    required: N	default: 0 0 0 	theta phi psi in degrees
 - -deformation   type: double(s)    required: N	default: 1 1 1 0 0 0 	xx yy zz xy yz xz


In [23]:
tv_ar = dtitk.registration.affSymTensor3DVolTask()
tv_ar.inputs.in_file = moving_tensor_new
tv_ar.inputs.transform = 'Ttensor_resampled.aff'

tv_ar.run()

print(tv_ar.cmdline)

180326-12:38:58,605 interface INFO:
	 stdout 2018-03-26T12:38:58.605299:inFile = Ttensor_resampled.nii.gz
180326-12:38:58,607 interface INFO:
	 stdout 2018-03-26T12:38:58.605299:outFile = Ttensor_resampled_affxfmd.nii.gz
180326-12:38:58,609 interface INFO:
	 stdout 2018-03-26T12:38:58.605299:interpolation option is LEI
180326-12:38:58,611 interface INFO:
	 stdout 2018-03-26T12:38:58.605299:reorientOption = PPD
180326-12:38:58,670 interface INFO:
	 stdout 2018-03-26T12:38:58.670448:Reading Ttensor_resampled.nii.gz ... Done in 0.062568s
180326-12:38:58,859 interface INFO:
	 stdout 2018-03-26T12:38:58.859287:trans (inverse) applied = [30.196, -9.065, 16.000]
180326-12:38:58,860 interface INFO:
	 stdout 2018-03-26T12:38:58.859287:[ 1.017, -0.137, -0.020;  0.141,  0.990,  0.132; -0.003, -0.143,  1.037]
180326-12:38:59,362 interface INFO:
	 stdout 2018-03-26T12:38:59.362583:backward resampling ...time consumed = 0.340
180326-12:38:59,988 interface INFO:
	 stdout 2018-03-26T12:38:59.988231:Wr

## Apply affine to Scalar Volume

### affineScalarVolume

Usage of affineScalarVolume : 
 - -in            type: string(s)    required: Y	
 - -out           type: string(s)    required: Y	
 - -trans         type: string(s)    required: N	specify an input transformation file; parameters input will be ignored
 - -interp        type: integer(s)   required: N	default: 0 	specify an interpolation scheme, 0 for trilinear (default) and 1 for nearest neighbor
 - -target        type: string(s)    required: N	output volume specification read from the target volume if specified
 - -translation   type: double(s)    required: N	default: 0 0 0 	x y z in mm
 - -euler         type: double(s)    required: N	default: 0 0 0 	theta phi psi in degrees
 - -deformation   type: double(s)    required: N	default: 1 1 1 0 0 0 	xx yy zz xy yz xz


In [24]:
sv_ar = dtitk.registration.affScalarVolTask()
sv_ar.inputs.in_file = moving_tr_new
sv_ar.inputs.transform = 'Ttensor_resampled.aff'

sv_ar.run()

print(sv_ar.cmdline)

180326-12:39:00,180 interface INFO:
	 stdout 2018-03-26T12:39:00.180009:inFile = Ttensor_resampled_tr.nii.gz
180326-12:39:00,184 interface INFO:
	 stdout 2018-03-26T12:39:00.180009:outFile = Ttensor_resampled_tr_affxfmd.nii.gz
180326-12:39:00,193 interface INFO:
	 stdout 2018-03-26T12:39:00.193048:Reading Ttensor_resampled_tr.nii.gz ... Done in 0.010675s
180326-12:39:00,223 interface INFO:
	 stdout 2018-03-26T12:39:00.223111:trans (inverse) applied = [30.196, -9.065, 16.000]
180326-12:39:00,224 interface INFO:
	 stdout 2018-03-26T12:39:00.223111:[ 1.017, -0.137, -0.020;  0.141,  0.990,  0.132; -0.003, -0.143,  1.037]
180326-12:39:00,374 interface INFO:
	 stdout 2018-03-26T12:39:00.374158:backward resampling ...time consumed = 0.100
180326-12:39:00,506 interface INFO:
	 stdout 2018-03-26T12:39:00.506769:Writing Ttensor_resampled_tr_affxfmd.nii.gz ... Done in 0.060s
affineScalarVolume -in Ttensor_resampled_tr.nii.gz -interp 0 -out Ttensor_resampled_tr_affxfmd.nii.gz -trans Ttensor_resamp

## Diffeomorphic Registration

### dti_diffeomorphic_reg

Output: Deformable alignment of a DTI volume (the subject) to a DTI template 

Usage: dti_diffeomorphic_reg template subject mask initial no_of_iter ftol

The required input are

- template: the filename of the template
- subject: the filename of the subject
- mask: the filename of a binary image that has 0 for background voxels and 1 for brain tissue voxels
- initial: a legacy parameter, just set to 1
- no_of_iter: number of iterations, recommended value to use is 6
- ftol: the minimum amount of change in the cost function as a fraction of the previous value at the previous iteration for the registration optimization to stop, recommended value is 0.002

The output from this command includes the registered image and the transformation that achieves the alignment. If we denote the filename of the subject as \${prefix}_aff.nii.gz, then the filename of the registered image will be \${prefix}_aff_diffeo.nii.gz and that of the transformation will be \${prefix}_aff_diffeo.df.nii.gz which is stored using the NIfTI vectorial format.

### Note

This is the point at which the dimension of the image matters (must be a power of 2). If you want to use linear registration only, then reslicing is not necessary. However, if you want to do nonlinear registration, then reslicing is required.

reslicing a mask didn't work for me... I had to create one in the new space using DTI-TK's tools (use TVtool to get trace, then put into BinaryThrehsoldImageFilter with params 0.01 100 1 0)

### Test interface

In [25]:
diffeo = dtitk.DiffeoTask()
diffeo.inputs.fixed_file = fixed_tensor_new
diffeo.inputs.moving_file = moving_tensor_new.replace('.nii.gz','_aff.nii.gz')
diffeo.inputs.mask_file = 'Ptensor_resampled_tr_thrbin.nii.gz'

diffeo_res = diffeo.run()

print(diffeo.cmdline)

180326-12:39:00,659 interface INFO:
	 stdout 2018-03-26T12:39:00.658761:registering Ttensor_resampled_aff.nii.gz to Ptensor_resampled.nii.gz ...
180326-12:39:00,664 interface INFO:
	 stdout 2018-03-26T12:39:00.664036:starting at Mon Mar 26 12:39:00 PDT 2018
180326-12:39:00,693 interface INFO:
	 stdout 2018-03-26T12:39:00.693774:
180326-12:39:00,696 interface INFO:
	 stdout 2018-03-26T12:39:00.693774:iteration 1 begins ...
180326-12:39:00,714 interface INFO:
	 stdout 2018-03-26T12:39:00.714599:Similarity Measure : DDS, Reorient Option : FS
180326-12:39:00,803 interface INFO:
	 stdout 2018-03-26T12:39:00.803649:Reading Ptensor_resampled.nii.gz ... Done in 0.075893s
180326-12:39:01,111 interface INFO:
	 stdout 2018-03-26T12:39:01.111196:Reading Ttensor_resampled_aff_diffeo.nii.gz ... Done in 0.059226s
180326-12:39:01,389 interface INFO:
	 stdout 2018-03-26T12:39:01.389785:Reading Ptensor_resampled_tr_thrbin.nii.gz ... Done in 0.004711s
180326-12:39:01,651 interface INFO:
	 stdout 2018-03-

180326-12:40:03,228 interface INFO:
	 stdout 2018-03-26T12:40:03.228191:iter 6: sum = 1250.89, img = 19076.4, prior = 534.516, reg = 1321.34
180326-12:40:05,991 interface INFO:
	 stdout 2018-03-26T12:40:05.991897:iter 7: sum = 1248.78, img = 19031.1, prior = 579.28, reg = 1224.71
180326-12:40:07,569 interface INFO:
	 stdout 2018-03-26T12:40:07.569093:iter = 65, iterGrad = 8
180326-12:40:07,570 interface INFO:
	 stdout 2018-03-26T12:40:07.569093:after : sum = 1247.68, img = 19000.3, prior = 577.277, reg = 1248.34
180326-12:40:07,571 interface INFO:
	 stdout 2018-03-26T12:40:07.571390:Little Endian System
180326-12:40:07,572 interface INFO:
	 stdout 2018-03-26T12:40:07.571390:Writing VECTORS ...
180326-12:40:08,395 interface INFO:
	 stdout 2018-03-26T12:40:08.395512:size is currently determined internally.
180326-12:40:08,396 interface INFO:
	 stdout 2018-03-26T12:40:08.395512:vsize = 1, 1, 1
180326-12:40:08,397 interface INFO:
	 stdout 2018-03-26T12:40:08.395512:origin is currently defa

180326-12:41:54,24 interface INFO:
	 stdout 2018-03-26T12:41:54.024094:iter 8: sum = 1154.11, img = 18605.1, prior = 154.447, reg = 663.982
180326-12:41:55,4 interface INFO:
	 stdout 2018-03-26T12:41:55.004846:iter 9: sum = 1152.71, img = 18696.3, prior = 142.812, reg = 502.139
180326-12:41:56,277 interface INFO:
	 stdout 2018-03-26T12:41:56.277110:iter = 67, iterGrad = 10
180326-12:41:56,279 interface INFO:
	 stdout 2018-03-26T12:41:56.277110:after : sum = 1152.38, img = 18725.8, prior = 132.846, reg = 468.672
180326-12:41:56,281 interface INFO:
	 stdout 2018-03-26T12:41:56.277110:Level 4
180326-12:41:56,285 interface INFO:
	 stdout 2018-03-26T12:41:56.285312:Piecewise Affine Setup: size = 32, 32, 16; vsize = 8, 8, 8
180326-12:41:56,291 interface INFO:
	 stdout 2018-03-26T12:41:56.291474:pDim = 196608; scalings : img = 0.06, prior = 0.09, reg = 0.036; ftol = 0.002
180326-12:41:56,534 interface INFO:
	 stdout 2018-03-26T12:41:56.534309:before : sum = 1152.38, img = 18724.4, prior = 133

180326-12:48:02,481 interface INFO:
	 stdout 2018-03-26T12:48:02.480955:Reading Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz ... Done in 0.710736s
180326-12:48:12,243 interface INFO:
	 stdout 2018-03-26T12:48:12.243418:backward resampling ...time consumed = 3.34374
180326-12:48:13,648 interface INFO:
	 stdout 2018-03-26T12:48:13.648132:Writing Ttensor_resampled_aff_diffeo_current.nii.gz ... Done in 0.698692s
180326-12:48:15,956 interface INFO:
	 stdout 2018-03-26T12:48:15.956009:IMAGE SIMILARITY: after previous iteration = 20922.8 after current iteration = 18651.9
180326-12:48:15,994 interface INFO:
	 stdout 2018-03-26T12:48:15.994555:iteration 2 done
180326-12:48:15,996 interface INFO:
	 stdout 2018-03-26T12:48:15.994555:
180326-12:48:15,998 interface INFO:
	 stdout 2018-03-26T12:48:15.994555:
180326-12:48:16,0 interface INFO:
	 stdout 2018-03-26T12:48:15.994555:iteration 3 begins ...
180326-12:48:16,24 interface INFO:
	 stdout 2018-03-26T12:48:16.024180:Similarity Measure :

180326-12:49:38,566 interface INFO:
	 stdout 2018-03-26T12:49:38.566116:. 
180326-12:49:42,292 interface INFO:
	 stdout 2018-03-26T12:49:42.292644:Writing Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz ... Done in 2.76s
180326-12:49:42,474 interface INFO:
	 stdout 2018-03-26T12:49:42.474494:computing the deformation field: d2*d1: x -> d1(x) + d2(x + d1(x))
180326-12:49:42,970 interface INFO:
	 stdout 2018-03-26T12:49:42.970223:Reading Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz ... Done in 0.46863s
180326-12:49:44,590 interface INFO:
	 stdout 2018-03-26T12:49:44.590698:Reading Ttensor_resampled_aff_diffeo.df.nii.gz ... Done in 0.649127s
180326-12:49:57,442 interface INFO:
	 stdout 2018-03-26T12:49:57.442473:Writing Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz ... Done in 5.77796s
180326-12:49:57,678 interface INFO:
	 stdout 2018-03-26T12:49:57.678209:inFile = Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz
180326-12:49:58,443 interface INFO:
	 stdout

180326-12:51:50,541 interface INFO:
	 stdout 2018-03-26T12:51:50.541228:Writing Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz ... Done in 1.70233s
180326-12:51:50,631 interface INFO:
	 stdout 2018-03-26T12:51:50.631616:converting to the diffeomorphic deformation field: ...
180326-12:51:50,989 interface INFO:
	 stdout 2018-03-26T12:51:50.989647:Reading Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz ... Done in 0.356019s
180326-12:51:53,785 interface INFO:
	 stdout 2018-03-26T12:51:53.785231:Gaussian smoothing: sigma = [   1,    1,    1] ... Done in 1.9s
180326-12:51:55,624 interface INFO:
	 stdout 2018-03-26T12:51:55.624592:Gaussian smoothing: sigma = [   1,    1,    1] ... Done in 1.81s
180326-12:51:55,665 interface INFO:
	 stdout 2018-03-26T12:51:55.665603:maxNorm = 0.55	iterations = 1
180326-12:51:55,708 interface INFO:
	 stdout 2018-03-26T12:51:55.708670:Voxelwise scaling Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz by 0.5 ... Done in 0.0427s
180326-12:51:

180326-12:53:09,956 interface INFO:
	 stdout 2018-03-26T12:53:09.956628:Little Endian System
180326-12:53:09,957 interface INFO:
	 stdout 2018-03-26T12:53:09.956628:Writing VECTORS ...
180326-12:53:10,355 interface INFO:
	 stdout 2018-03-26T12:53:10.355539:size is currently determined internally.
180326-12:53:10,356 interface INFO:
	 stdout 2018-03-26T12:53:10.355539:vsize = 1, 1, 1
180326-12:53:10,357 interface INFO:
	 stdout 2018-03-26T12:53:10.355539:origin is currently default to [0, 0, 0].
180326-12:53:10,359 interface INFO:
	 stdout 2018-03-26T12:53:10.355539:outFile = Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz
180326-12:53:10,361 interface INFO:
	 stdout 2018-03-26T12:53:10.361072:Little Endian System
180326-12:53:10,362 interface INFO:
	 stdout 2018-03-26T12:53:10.361072:Reading VECTORS ...
180326-12:53:16,454 interface INFO:
	 stdout 2018-03-26T12:53:16.454667:Writing Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz ... Done in 1.5143s
180326-12:53:16,528 int

180326-12:54:44,122 interface INFO:
	 stdout 2018-03-26T12:54:44.121336:after : sum = 837.154, img = 13334.3, prior = 276.489, reg = 1163.53
180326-12:54:44,124 interface INFO:
	 stdout 2018-03-26T12:54:44.124104:Little Endian System
180326-12:54:44,132 interface INFO:
	 stdout 2018-03-26T12:54:44.124104:Writing VECTORS ...
180326-12:54:44,534 interface INFO:
	 stdout 2018-03-26T12:54:44.534094:size is currently determined internally.
180326-12:54:44,535 interface INFO:
	 stdout 2018-03-26T12:54:44.534094:vsize = 1, 1, 1
180326-12:54:44,536 interface INFO:
	 stdout 2018-03-26T12:54:44.534094:origin is currently default to [0, 0, 0].
180326-12:54:44,537 interface INFO:
	 stdout 2018-03-26T12:54:44.534094:outFile = Ttensor_resampled_aff_to_Ptensor_resampled.5.df.nii.gz
180326-12:54:44,539 interface INFO:
	 stdout 2018-03-26T12:54:44.539089:Little Endian System
180326-12:54:44,540 interface INFO:
	 stdout 2018-03-26T12:54:44.539089:Reading VECTORS ...
180326-12:54:50,304 interface INFO:
	

## Combine Transforms: Affine and Diffeomorphic 

### dfRightComposeAffine

Usage of dfRightComposeAffine :

 - -df            type: string(s)    required: Y	
 - -aff           type: string(s)    required: Y	
 - -out           type: string(s)    required: Y	

### Test interface

In [26]:
combo = dtitk.ComposeXfmTask()
combo.inputs.in_df = 'Ttensor_resampled_aff_diffeo.df.nii.gz'
combo.inputs.in_aff = 'Ttensor_resampled.aff'

combo_res = combo.run()

print(combo.cmdline)

180326-12:55:46,718 interface INFO:
	 stdout 2018-03-26T12:55:46.718682:computing the deformation field: aff*df: x -> aff(x+df(x)) - x
180326-12:55:47,452 interface INFO:
	 stdout 2018-03-26T12:55:47.452875:Reading Ttensor_resampled_aff_diffeo.df.nii.gz ... Done in 0.667566s
180326-12:55:56,826 interface INFO:
	 stdout 2018-03-26T12:55:56.825846:Writing Ttensor_resampled_aff_diffeo.df_aff.df.nii.gz ... Done in 5.19024s
dfRightComposeAffine -aff Ttensor_resampled.aff -df Ttensor_resampled_aff_diffeo.df.nii.gz -out Ttensor_resampled_aff_diffeo.df_aff.df.nii.gz


## Apply combined diffeo to tensor

### deformationSymTensor3DVolume

Usage of deformationSymTensor3DVolume : 

 - -in            type: string(s)    required: Y	
 - -out           type: string(s)    required: Y	
 - -trans         type: string(s)    required: Y	
 - -df            type: string(s)    required: N	default: FD 
 - -interp        type: string(s)    required: N	default: LEI 	EI or LEI
 - -reorient      type: string(s)    required: N	default: PPD 	FS or PPD
 - -target        type: string(s)    required: N	output volume specification read from the target volume if specified
 - -vsize         type: double(s)    required: N	
 - -flip          type: integer(s)   required: N	default: 0 0 0 
 - -type          type: integer(s)   required: N	default: 1 	resampling type, 1 for backward (default) and 0 for forward


In [27]:
tv_applydiff = dtitk.registration.diffeoSymTensor3DVolTask()
tv_applydiff.inputs.in_file = moving_tensor_new
tv_applydiff.inputs.transform = 'Ttensor_resampled_aff_diffeo.df_aff.df.nii.gz'

tv_applydiff.run()

print(tv_applydiff.cmdline)

180326-12:55:57,26 interface INFO:
	 stdout 2018-03-26T12:55:57.026594:inFile = Ttensor_resampled.nii.gz
180326-12:55:57,31 interface INFO:
	 stdout 2018-03-26T12:55:57.026594:outFile = Ttensor_resampled_diffeoxfmd.nii.gz
180326-12:55:57,32 interface INFO:
	 stdout 2018-03-26T12:55:57.026594:transFile = Ttensor_resampled_aff_diffeo.df_aff.df.nii.gz
180326-12:55:57,34 interface INFO:
	 stdout 2018-03-26T12:55:57.026594:df option is FD
180326-12:55:57,36 interface INFO:
	 stdout 2018-03-26T12:55:57.026594:interpolation option is LEI
180326-12:55:57,37 interface INFO:
	 stdout 2018-03-26T12:55:57.026594:reorientOption = PPD
180326-12:55:57,95 interface INFO:
	 stdout 2018-03-26T12:55:57.095839:Reading Ttensor_resampled.nii.gz ... Done in 0.066703s
180326-12:55:57,908 interface INFO:
	 stdout 2018-03-26T12:55:57.908688:Reading Ttensor_resampled_aff_diffeo.df_aff.df.nii.gz ... Done in 0.615749s
180326-12:56:00,907 interface INFO:
	 stdout 2018-03-26T12:56:00.907724:backward resampling ...ti

## Apply combined diffeo to scalar volume

### deformationScalarVolume
Usage of deformationScalarVolume :

 - -in            type: string(s)    required: Y	
 - -out           type: string(s)    required: Y	
 - -trans         type: string(s)    required: Y	
 - -target        type: string(s)    required: N	output volume specification read from the target volume if specified
 - -vsize         type: double(s)    required: N	
 - -flip          type: integer(s)   required: N	default: 0 0 0 
 - -type          type: integer(s)   required: N	default: 1 	resampling type, 1 for backward (default) and 0 for forward
 - -interp        type: integer(s)   required: N	default: 0 	interpolation type, 0 for trilinear (default) and 1 for nearest neighbor

In [28]:
sv_applydiff = dtitk.registration.diffeoScalarVolTask()
sv_applydiff.inputs.in_file = moving_tr_new
sv_applydiff.inputs.transform = 'Ttensor_resampled_aff_diffeo.df_aff.df.nii.gz'

sv_applydiff.run()

print(sv_applydiff.cmdline)

180326-12:56:02,180 interface INFO:
	 stdout 2018-03-26T12:56:02.180769:inFile = Ttensor_resampled_tr.nii.gz
180326-12:56:02,183 interface INFO:
	 stdout 2018-03-26T12:56:02.180769:outFile = Ttensor_resampled_tr_diffeoxfmd.nii.gz
180326-12:56:02,185 interface INFO:
	 stdout 2018-03-26T12:56:02.180769:transFile = Ttensor_resampled_aff_diffeo.df_aff.df.nii.gz
180326-12:56:02,194 interface INFO:
	 stdout 2018-03-26T12:56:02.194754:Reading Ttensor_resampled_tr.nii.gz ... Done in 0.011791s
180326-12:56:02,847 interface INFO:
	 stdout 2018-03-26T12:56:02.847361:Reading Ttensor_resampled_aff_diffeo.df_aff.df.nii.gz ... Done in 0.617856s
180326-12:56:03,844 interface INFO:
	 stdout 2018-03-26T12:56:03.844860:backward resampling ...time consumed = 0.212678
180326-12:56:03,925 interface INFO:
	 stdout 2018-03-26T12:56:03.925737:Writing Ttensor_resampled_tr_diffeoxfmd.nii.gz ... Done in 0.045478s
deformationScalarVolume -in Ttensor_resampled_tr.nii.gz -interp 0 -out Ttensor_resampled_tr_diffeoxfm